<a href="https://colab.research.google.com/github/Viviancs/torch_exercise/blob/main/test_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1.数据预处理

In [ ]:
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

In [ ]:
import torch
print(torch.__version__)


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.utils.data import DataLoader,Dataset,TensorDataset

In [3]:
import os
os.chdir("./drive/MyDrive/pytorch_ex")

In [ ]:
dftrain_raw=pd.read_csv('./data/titanic/train.csv')
dftest_raw=pd.read_csv('./data/titanic/test.csv')
dftrain_raw.head((10))

In [ ]:
ax=dftrain_raw['Survived'].value_counts().plot(kind='bar',figsize=(12,8),fontsize=15,rot=0)
ax.set_ylabel('Counts',fontsize=15)
ax.set_xlabel('Survived',fontsize=15)
plt.show()

In [ ]:
ax=dftrain_raw['Age'].plot(kind='hist',bins=20,figsize=(12,8),fontsize=15)
ax.set_ylabel('Frequency',fontsize=15)
ax.set_xlabel('Age',fontsize=15)
plt.show()

In [ ]:
ax=dftrain_raw.query('Survived==0')['Age'].plot(kind='density',figsize=(12,8),fontsize=15)
dftrain_raw.query('Survived==1')['Age'].plot(kind='density',figsize=(12,8),fontsize=15)
ax.set_ylabel('Density',fontsize=15)
ax.set_xlabel('Age',fontsize=15)
plt.show()

In [ ]:
dfresult=pd.DataFrame()

#Pclass one-hot编码
dfPclass=pd.get_dummies(dftrain_raw['Pclass'])
dfPclass.columns=['Pclass_'+str(x) for x in dfPclass.columns]
dfresult=pd.concat([dfresult,dfPclass],axis=1)
print(dfPclass)


#Sex
dfSex=pd.get_dummies(dftrain_raw['Sex'])
dfresult=pd.concat([dfresult,dfSex],axis=1)

#Age
dfresult['Age']=dftrain_raw['Age'].fillna(0)
dfresult['Age_null']=pd.isna(dftrain_raw['Age']).astype('int32')


#sibsp #parch fare 
dfresult['SibSp']=dfftest_raw['SibSp']
dfresult['Parch']=dfftest_raw['Parch']
dfresult['Fare']=dfftest_raw['Fare']
#cabin 
dfresult['Cabin']=dftrain_raw['Cabin'].fillna(0)
dfresult['Cabin_null']=pd.isna(dftrain_raw['Cabin']).astype('int32')

#embarked
dfEmbarked=pd.get_dummies(dftrain_raw['Embarked'],dummy_na=True)
dfEmbarked.columns=['Embark_'+str(x) for x in dfEmbarked.columns]
dfresult=pd.concat([dfresult,dfEmbarked],axis=1)
print(dfresult)

In [29]:
##数据预处理
def preprocess(dfdata):
  dfresult=pd.DataFrame()

  #Pclass one-hot编码
  dfPclass=pd.get_dummies(dfdata['Pclass'])
  dfPclass.columns=['Pclass_'+str(x) for x in dfPclass.columns]
  dfresult=pd.concat([dfresult,dfPclass],axis=1)

  #Sex
  dfSex=pd.get_dummies(dfdata['Sex'])
  dfresult=pd.concat([dfresult,dfSex],axis=1)

  #Age
  dfresult['Age']=dfdata['Age'].fillna(0)
  dfresult['Age_null']=pd.isna(dfdata['Age']).astype('int32')

  #sibsp parch fare
  dfresult['SibSp']=dfdata['SibSp']
  dfresult['Parch']=dfdata['Parch']
  dfresult['Fare']=dfdata['Fare']
  #sibsp #parch fare 
  dfresult['SibSp']=dfdata['SibSp']
  dfresult['Parch']=dfdata['Parch']
  dfresult['Fare']=dfdata['Fare']
  #cabin 
  dfresult['Cabin_null']=pd.isna(dfdata['Cabin']).astype('int32')

  #embarked
  dfEmbarked=pd.get_dummies(dfdata['Embarked'],dummy_na=True)
  dfEmbarked.columns=['Embark_'+str(x) for x in dfEmbarked.columns]
  dfresult=pd.concat([dfresult,dfEmbarked],axis=1)

  return(dfresult)

x_train=preprocess(dftrain_raw).values
y_train=dftrain_raw[['Survived']].values

x_test=preprocess(dftest_raw).values
y_test=dftest_raw[['Survived']].values

print('x_train.shape=',x_train.shape)
print('x_test.shape=',x_test.shape)

print('y_train.shape=',y_train.shape)
print('y_test.shape=',y_test.shape)

x_train.shape= (712, 15)
x_test.shape= (179, 15)
y_train.shape= (712, 1)
y_test.shape= (179, 1)


In [30]:
d1_train=DataLoader(TensorDataset(torch.tensor(x_train).float(),torch.tensor(y_train).float()),shuffle=True,batch_size=8)
d1_test=DataLoader(TensorDataset(torch.tensor(x_test).float(),torch.tensor(y_test).float()),shuffle=False,batch_size=8)


In [ ]:
for features,labels in d1_train:
  print(features,labels)

## 2.定义模型

In [49]:
def create_net():
  net=nn.Sequential()
  net.add_module('linear1',nn.Linear(15,20))
  net.add_module('relu1',nn.ReLU())
  net.add_module('linear2',nn.Linear(20,15))
  net.add_module('relu2',nn.ReLU())
  net.add_module('linear3',nn.Linear(15,1))
  net.add_module('sigmoid',nn.Sigmoid())
  return net

net=create_net()
print(net)

Sequential(
  (linear1): Linear(in_features=15, out_features=20, bias=True)
  (relu1): ReLU()
  (linear2): Linear(in_features=20, out_features=15, bias=True)
  (relu2): ReLU()
  (linear3): Linear(in_features=15, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)


In [50]:
from torchkeras import summary
summary(net,input_shape=(15,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 20]             320
              ReLU-2                   [-1, 20]               0
            Linear-3                   [-1, 15]             315
              ReLU-4                   [-1, 15]               0
            Linear-5                    [-1, 1]              16
           Sigmoid-6                    [-1, 1]               0
Total params: 651
Trainable params: 651
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.000057
Forward/backward pass size (MB): 0.000549
Params size (MB): 0.002483
Estimated Total Size (MB): 0.003090
----------------------------------------------------------------


## 3.训练模型
